In [1]:
!pip install mrjob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 KB 2.6 MB/s eta 0:00:00a 0:00:01


In [32]:
%%file join.py

import sys, os, re
from mrjob.job import MRJob

class MRJoin(MRJob):
  
  SORT_VALUES = True
  
  def mapper(self, _, line):    
    splits = line.rstrip("\n").split("|")
    
    if len(splits) == 2: # user data
      name = splits[0]
      team_id = splits[1]
      yield team_id, ("user", name)
        
    else: # country data
      team_name = splits[0]
      country_index = splits[1]
      team_id = splits[2]
      yield team_id, ("team", team_name, country_index)

  def reducer(self, key, values):
    values = [x for x in values]
    team_data = ()        
    
    for x in values:
      if x[0] == "team":
          team_data = x
          break

    if team_data is ():
        pass
    else:
        for x in values:
            if x[0] == "user":
                yield(None, (x[1], team_data[1], team_data[2]))
      
if __name__ == '__main__':
  MRJoin.run()

Overwriting join.py


In [33]:
%%file teams.txt
Telepuziki|US|1001
Romashka|CA|1002
Zaichiki|UK|1004
Lapsha|IT|1200
Medvejata|RU|2321

Overwriting teams.txt


In [34]:
%%file users.txt
Alice Bob|1001
Sam Sneed|1002
Jon Sneed|1002
Arnold Wesise|1004
Alice Wesise|1004
Henry Bob|1001
Yo Yo Ma|1006
Jon York|1002
Alex Ball|1004
Jim Davis|1008
A Chou|1008

Overwriting users.txt


In [36]:
!python3 join.py teams.txt users.txt

/opt/join.py:32: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if team_data is ():
No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/join.root.20231115.101140.501302
Running step 1 of 1...
job output is in /tmp/join.root.20231115.101140.501302/output
Streaming final output from /tmp/join.root.20231115.101140.501302/output...
null	["Jon Sneed", "Romashka", "CA"]
null	["Jon York", "Romashka", "CA"]
null	["Sam Sneed", "Romashka", "CA"]
null	["Alex Ball", "Zaichiki", "UK"]
null	["Alice Wesise", "Zaichiki", "UK"]
null	["Arnold Wesise", "Zaichiki", "UK"]
null	["Alice Bob", "Telepuziki", "US"]
null	["Henry Bob", "Telepuziki", "US"]
Removing temp directory /tmp/join.root.20231115.101140.501302...


In [19]:
!hadoop fs -put countries.txt /externalData

In [20]:
!hadoop fs -put users.txt /externalData